# Mapping federal crop insurance in the U.S.

A Jupyter notebook (Python 3) by Peter Donovan, info@soilcarboncoalition.org

Open data is not just a thing or a tool. It's a behavior, based on beliefs. This notebook is a way of sharing my methods and assumptions, and if you use the same or similar tools (such as R instead of Python, for example) you can retread these steps. I hope this notebook may also serve as a guide for me as well as others who want to do similar things.

With crop insurance, as with any data set, looking at the data is a good way of learning about its particulars if not its intentions. Some knowledge of the context or domain of the data is usually required. 

For background on federal crop insurance, the following may be a start:

Dennis Shields' 2015 report from the Congressional Research Service: https://fas.org/sgp/crs/misc/R40532.pdf

Environmental Working Group's material on crop insurance, which includes interactive maps showing rate of return (payouts compared to premiums) on some crops by county from 2001 through 2014: http://www.ewg.org/research/crop-insurance-lottery. The average federal subsidy for crop insurance premiums is about 60%.

The Natural Resources Defense Council has a 2013 paper on crop insurance, https://www.nrdc.org/sites/default/files/soil-matters-IP.pdf. This paper suggests that crop insurance could be reformed to reward farming that is low risk with environmental rewards.

A starting hypothesis: federally subsidized crop insurance, while it sustains the economic viability of many farm businesses, might also tend to replace soil health and function as the foundation of a viable agriculture.

To investigate the hypothesis, we'll start by compiling data.

First, we get data. Download and unzip the data file from the USDA Risk Management Agency website: http://www.rma.usda.gov/data/cause.html The complete data for each year is under the "Summary of Business with Month of Loss" header.  So far I am using the 2014 through 2016 data. You can get the column headers from the same web page as a Word or pdf doc.

In [1]:
#some usual imports, including some options for displaying large currency amounts with commas and only 2 decimals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.float_format', '{:,}'.format)
pd.set_option('display.precision',2)

From http://www.rma.usda.gov/data/cause we see that years 2010 through 2016 are available as zip archives in Summary of Business. With a slower connection it is better to download and extract the zip archives outside of this notebook. Each contains a text file such as colsom14.txt, which will be an example for this notebook.

Unzip the file and inspect it with a text editor. There are pipe characters separating the fields, and sometimes sequences of spaces before them or after them. There are no column headers, we'll add those next.

In [4]:
df = pd.read_csv('/Users/Peter/Documents/atlas/RMA/colsom14.txt',sep='|', )
df.shape #this counts rows, columns in our dataframe

(133978, 24)

In [5]:
df.columns

Index(['2014', '01', 'AL', '001', 'Autauga                       ', '0021',
       'COTTON                        ', '03', 'RPHPE     ', 'A', 'H ', '01.1',
       'Decline in Price                   ', '11', '   ', '1', '1.1',
       '132.68', '50069', '7199', '4247', '132.68.1', '6294', '.87'],
      dtype='object')

In [24]:
df16 = pd.read_csv('/Users/Peter/Documents/atlas/RMA/colsom16.txt',sep='|',header=None)
df16.shape #this counts rows, columns in our dataframe

(105213, 25)

The column headers are supplied in a Word document (Record layout: Word) from the same web page. They differ for 2010-2014 and from 2015 forward. Format them as a python list of strings as follows, and add them to the dataframe.

In [25]:
the_columns_2014 = ['Crop Year Identifier','State Code','State Abbreviation ','County Code','County Name','Crop Code','Crop Name','Insurance Plan Code','Insurance Plan Name Abbreviation','Coverage Category','Stage Code','Cause of Loss Code','Cause of Loss Description','Month of Loss','Month of Loss Name','Policies Earning Premium','Policies Indemnified','Net Planted Acres','Liability','Total Premium','Subsidy','Determined Acres','Indemnity Amount','Loss Ratio']



the_columns_15_16 = ['Crop Year Identifier', 'State Code', 'State Abbreviation ',
       'County Code', 'County Name', 'Crop Code', 'Crop Name',
       'Insurance Plan Code', 'Insurance Plan Name Abbreviation',
       'Coverage Category', 'Stage Code', 'Cause of Loss Code',
       'Cause of Loss Description', 'Month of Loss', 'Month of Loss Name',
       'Policies Earning Premium', 'Policies Indemnified', 'Net Planted Acres',
       'Net Endorsed Acres', 'Liability', 'Total Premium', 'Subsidy',
       'Determined Acres', 'Indemnity Amount', 'Loss Ratio']
df16.columns = the_columns_15_16 #this adds our column headers

There are spaces on either side of some of the fields. We can use str.strip() to remove them. 

In [26]:
#we strip excess white space from the columns (numeric columns don't work for strip)
cols_w_spaces = ['County Name','Crop Name','Insurance Plan Name Abbreviation','Cause of Loss Description']

for item in cols_w_spaces:
    df16[item] = df16[item].map(lambda x: x.strip())
#check the result
print(list(df16.loc[1187]))

[2016, 1, 'AL', 83, 'Limestone', 41, 'CORN', 2, 'RP', 'A', 'R ', '44', 'Cold Wet Weather', 5, 'MAY', 1, 1, 7.8399999999999999, 0.0, 2069.0999999999999, 181.5, 107.25, 7.8399999999999999, 238.69999999999999, 1.3200000000000001]


### FIPS code
The state and county location codes are numeric (int64). FIPS (Federal Information Processing Standard) codes for counties are 5-digit strings. We'll pad with zeros using zfill function. This will come in handy when it comes to mapping, as we will want to merge or join our data with county boundaries using the FIPS code.

In [27]:
df = df16

In [3]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23],
           dtype='int64')

In [28]:
#convert to strings, pad with zeros, 2 digits for state, 3 for county
df['State Code'] = df['State Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(2))
df['County Code'] = df['County Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(3))

#add FIPS or id column and test
df['FIPS'] = df['State Code'] + df['County Code']
df['FIPS'][10] #to make sure we have a 5-digit string, not a number

'01001'

In [33]:
corn_acres['State County Code']=corn_acres['State County Code'].map(lambda x: str(x)).apply(lambda x: x.zfill(5))

In [43]:
corn_by_acre = pd.concat([corn,corn_acres],axis=1)
corn_by_acre.shape

(2620, 3)

In [41]:
corn.reset_index('County Name', inplace=True)

In [45]:
corn_by_acre['Indemnity'] = corn_by_acre['Indemnity Amount']/corn_by_acre['acres']

In [49]:
corn_by_acre.max()

Indemnity Amount   12,320,939.000000002
acres                        356,670.96
Indemnity             400.6666666666667
dtype: float64

## Map indemnities by crop and county acreage

In [12]:
corn = df[df['Crop Name']=='CORN'].groupby(['FIPS','County Name']).agg({'Indemnity Amount':np.sum}); corn.shape

(1762, 1)

In [16]:
corn.mean()

Indemnity Amount   531,034.2014755959
dtype: float64

## Map indemnities by county

In [6]:
counties = df.groupby(['FIPS','County Name'])
aggregated = counties.agg({'Indemnity Amount': np.sum})
aggregated.sort_values('Indemnity Amount',ascending=False)

,,Indemnity Amount
FIPS,County Name,
06019,Fresno,105742443
48165,Gaines,71305188
27129,Renville,57776883
06999,All Other Counties,49644074
27143,Sibley,49374967
06029,Kern,49220195
19155,Pottawattamie,48536658
40065,Jackson,45532816
19109,Kossuth,44277075


In [7]:
aggregated.reset_index(level=0, inplace=True)
aggregated.reset_index(level=0, inplace=True)
#run this twice to convert the two indexes to columns

In [8]:
#rename columns for convenience
aggregated.rename(columns={'County Name': 'name', 'FIPS': 'id', 'Indemnity Amount': 'indemnity'}, inplace=True)
#convert to $millions
aggregated['indemnity']=aggregated['indemnity']/1000000
#reorder columns and write to tab-separated tsv file for d3 mapping
aggregated = aggregated[['id','name','indemnity']]
aggregated.to_csv('/Users/Peter/Documents/atlas/RMA/indemnity2014.tsv', sep='\t', index=False)

## Causes of loss
Let's look at the causes of loss. NOTE: These procedures could be duplicated to aggregate indemnities by 'Crop Name' as well.

In [9]:
df.groupby('Cause of Loss Description').agg({'Indemnity Amount':np.sum}).sort_values('Indemnity Amount',ascending=False)

,Indemnity Amount
Cause of Loss Description,
Excess Moisture/Precip/Rain,2483884562
Decline in Price,1848696139
Drought,1718087747
Hail,933202124
Cold Wet Weather,339836298
Area Plan Crops Only,316749929
Failure Irrig Supply,311330493
Freeze,273106811
Wind/Excess Wind,220228074


In [10]:
causes_2014 = df.groupby('Cause of Loss Description')['Indemnity Amount'].sum()
causes_2014.sort_values(ascending=False)

Cause of Loss Description
Excess Moisture/Precip/Rain          2483884562
Decline in Price                     1848696139
Drought                              1718087747
Hail                                  933202124
Cold Wet Weather                      339836298
Area Plan Crops Only                  316749929
Failure Irrig Supply                  311330493
Freeze                                273106811
Wind/Excess Wind                      220228074
Heat                                  203128308
Cold Winter                           109445405
Frost                                  78652859
Flood                                  75085010
Plant Disease                          59895203
Other (Snow-Lightning-Etc.)            45130916
Hot Wind                               40752799
Mycotoxin (Aflatoxin)                  28388250
Wildlife                               18875194
Hurricane/Tropical Depression           7931157
Insects                                 4835435
Failure Irrig 

In [11]:
#to generate a table of total indemnities by Cause of Loss, you can export a csv
causes_2014.to_csv('/Users/Peter/Documents/atlas/RMA/causes_2014.csv')

'Excess Moisture/Precip/Rain' and 'Drought' are by far the most common causes. Let's filter the dataframe by these two, so we can potentially see which counties had indemnities for both causes, and how much.

In [12]:
rain = df[df['Cause of Loss Description']=='Excess Moisture/Precip/Rain']
drought = df[df['Cause of Loss Description']=='Drought']
print(rain.shape, drought.shape)

(37767, 25) (22937, 25)


Now do a groupby on each dataframe by county, with sums of indemnity amounts.

In [13]:
g_rain = rain.groupby(['FIPS','County Name']).agg({'Indemnity Amount':np.sum})
g_drought = drought.groupby(['FIPS','County Name']).agg({'Indemnity Amount':np.sum})
together=pd.concat([g_rain,g_drought],axis=1)
together.columns = ['moisture','drought']
together.head()

,,moisture,drought
FIPS,County Name,,
01001,Autauga,"2,442.0","18,849.0"
01003,Baldwin,"816,341.0","116,282.0"
01005,Barbour,"20,835.0","588,264.0"
01007,Bibb,"22,572.0","43,286.0"
01011,Bullock,"60,122.0","77,797.0"


Let's add two columns, a total, and a ratio of moisture to drought.

In [14]:
together['total']=together.moisture + together.drought
together['ratio']=together.moisture / together.drought
together.head(20)

,,moisture,drought,total,ratio
FIPS,County Name,,,,
01001,Autauga,"2,442.0","18,849.0","21,291.0",0.1295559446124463
01003,Baldwin,"816,341.0","116,282.0","932,623.0",7.0203556870366866
01005,Barbour,"20,835.0","588,264.0","609,099.0",0.0354177716127453
01007,Bibb,"22,572.0","43,286.0","65,858.0",0.5214619045418842
01011,Bullock,"60,122.0","77,797.0","137,919.0",0.772806149337378
01013,Butler,"16,653.0","29,017.0","45,670.0",0.5739049522693593
01015,Calhoun,"36,100.0","15,935.0","52,035.0",2.265453404455601
01017,Chambers,nan,386.0,nan,nan
01019,Cherokee,"31,976.0","12,436.0","44,412.0",2.5712447732389836


In [15]:
mixed = together[(together.ratio < 4) & (together.ratio > .25)]
mixed.shape

(513, 4)

In [16]:
mixed.reset_index(level=0, inplace=True)
mixed.reset_index(level=0, inplace=True)
#run this twice

In [17]:
mixed = mixed.rename(columns={'total':'indemnity'})

In [18]:
mixed.indemnity = mixed.indemnity/1000000

In [19]:
mixed.to_csv('/Users/Peter/Documents/atlas/RMA/moisture_plus_drought_2014.tsv', sep='\t', index=False)

## Crop acreage
We may want to look at indemnities related to crop acreage in a county. The Farm Service Administration keeps track of cropland acreage. Data is available by year at https://www.fsa.usda.gov/news-room/efoia/electronic-reading-room/frequently-requested-information/crop-acreage-data/index. We'll start with 2016 crop year, aggregating acres by county.

In [17]:
df = pd.read_excel('/Users/Peter/Documents/atlas/RMA/2016_fsa_acres/2016_fsa_acres_010417.xlsx', sheetname='county_data')

In [21]:
df.columns

Index(['State Code', 'County Code', 'Crop Code', 'State', 'County',
       'State County Code', 'Crop', 'Crop Type', 'Intended Use',
       'Irrigation Practice', 'Planted Acres', 'Volunteer Acres',
       'Failed Acres', 'Prevented Acres', 'Not Planted Acres',
       'Planted and Failed Acres', 'acres'],
      dtype='object')

In [20]:
df['acres'] = df['Planted and Failed Acres']+df['Volunteer Acres']+df['Prevented Acres']

In [22]:
df = df[['State County Code','acres','Crop']]

In [23]:
corn_acres = df[df['Crop']=='CORN'].groupby('State County Code').agg({'acres':np.sum})

,acres
State County Code,
1001,"1,170.76"
1003,"7,606.080000000001"
1005,"2,350.97"
1007,27.2
1009,"1,587.3400000000001"
1011,571.47
1013,"1,463.1299999999999"
1015,840.98
1017,429.65999999999997


In [ ]:
corn_acres

In [26]:
df = df.rename(columns={'State County Code':'id'})

In [27]:
grouped = df.groupby('id').agg({'acres':np.sum})

In [58]:
grouped[grouped.acres > 4000000]

,acres
id,
4001,"4,847,170.78"
4005,"5,212,903.01"
32013,"6,103,906.630000002"
41025,"4,453,800.55"
56037,"4,478,592.130000001"


In [29]:
df2 = pd.read_csv('/Users/Peter/Documents/atlas/RMA/indemnity2016.tsv', sep='\t')
df2.set_index('id',inplace=True)
df2.shape

(2703, 2)

In [46]:
joined=pd.concat([df2,grouped],axis=1)

In [47]:
joined['ratio'] = joined.indemnity / joined['acres']*1000000

In [48]:
joined = joined.reset_index()

In [49]:
joined.id = joined.id.map(lambda x: str(x)).apply(lambda x: x.zfill(5))

In [50]:
joined = joined[['id','name','ratio']]

In [64]:
joined = joined.rename(columns={'index':'id'})

In [58]:
corn_by_acre = corn_by_acre.rename(columns={'index':'id'})

In [60]:

corn_by_acre[['id','name','indemnity']].to_csv('/Users/Peter/Documents/atlas/RMA/corn_by_acre.tsv',sep='\t')


In [61]:
corn_by_acre.indemnity.describe()

count                1,760.0
mean      17.857640729381185
std       27.673672436031378
min     0.029772117150757924
25%        2.699585695009991
50%        7.912187470332242
75%        21.73488160711495
max        400.6666666666667
Name: indemnity, dtype: float64

In [67]:
joined.indemnity.describe()

count               2,666.0
mean      18.36627861759955
std       166.9844856142665
min     0.00420908323753347
25%       1.553177194237957
50%       4.205278759792259
75%      12.999815226664863
max      6,980.731364275668
Name: indemnity, dtype: float64

In [66]:
joined.to_csv('/Users/Peter/Documents/atlas/RMA/indemnity_per_acre_2016.tsv', sep='\t')

In [14]:
cols_w_spaces = ['County Name','Crop Name','Insurance Plan Name Abbreviation','Cause of Loss Description']

for item in cols_w_spaces:
    df16[item] = df16[item].map(lambda x: x.strip())


In [15]:
df16g = df16.groupby('Crop Name').agg({'Indemnity Amount':np.sum})

In [20]:
df16g.sort_values('Indemnity Amount', ascending=False)

,Indemnity Amount
Crop Name,
CORN,"935,682,263.0000004"
WHEAT,"493,662,051.9999991"
SOYBEANS,"373,826,331.9999997"
COTTON,"302,464,387.00000024"
All Other Crops,"177,054,364.99999994"
"PASTURE,RANGELAND,FORAGE","143,857,349.0"
FLUE CURED TOBACCO,"126,479,742.00000006"
RICE,"104,175,766.0"
BURLEY TOBACCO,"84,075,077.0"


In [16]:
df16g[df16g['Crop Name']=='PASTURE, RANGELAND, FORAGE']

KeyError: 'Crop Name'